In [11]:
import os
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd

path = 'C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/ham_set'
files = os.listdir(path)
filenames = []

for root, dirs, files in os.walk(path):
  for file in files:
    if file.endswith('.csv'):
      filenames.append(os.path.join(root, file))

all_pre_dfs = []

# Iterate through the list of filenames and read each CSV file
for filename in filenames:
  pre_df = pd.read_csv(filename, sep='\t')
  pre_df.columns = ['data']
  pre_df['spam'] = 0

  # Create a list to store DataFrames
  dpre_df = dd.from_pandas(pre_df, npartitions=4) # Create dpre_df after modifications

  # Append the current DataFrame to the list
  result = dpre_df.compute()
  all_pre_dfs.append(result)

# Concatenate all DataFrames in the list into a single DataFrame
raw = pd.concat(all_pre_dfs, ignore_index=True)

In [ ]:
# 텍스트 처리 함수 설정
def preprocess_text(text):
    # 여러 개의 공백을 하나의 공백으로 축소
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    text = text.strip()
    # 맨 앞의 숫자와 공백 또는 콜론 제거
    text = re.sub(r'^\d+\s*', '', text)
    # 필요에 따라 특수 문자 제거 (예시로 일부 문자만 제거)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    return text

In [24]:
# 결측치 처리
print(len(raw))
drop_index = raw[raw['data'].isnull()].index
raw = raw.drop(drop_index)
print(len(raw))

40038
40038


In [26]:
# 중복 내용 제거
print(len(raw))
raw = raw.drop_duplicates('data')
print(len(raw))

40038
39721


In [25]:
raw['data']=raw['data'].apply(preprocess_text)

In [27]:
raw.head()

,data,spam
0,웅 자취비 장난 아님,0
1,키키 맞아,0
2,나도 지금 서울살이하고 있잖아,0
3,거기 보증금이랑 월세 얼마야,0
4,여기 되게 싸게 주고 왔어,0


In [28]:
raw.to_csv('C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/ham_data.csv', index=False)